https://docs.google.com/forms/d/e/1FAIpQLSfRHORtHFiPUGCiYNt2NfapWtgUQWbv5V75kUPwUAkx20r9Eg/viewform

"What was the total value of 'Cash and cash equivalents' as of December 31, 2023?"

"Who are Meta's 'Directors' (i.e., members of the Board of Directors)?"

1. Load data (pdf)
2. Process data - using llama parse (proprietary parsing w embedded object - tables and figures - exactly what i am looking for.) Build retrieval over complex and semi structured documents. - production grade context agumentation. (Suits well with my EOB parsing and loading project). Does it in markdown format - because easier to read the format and parse. Makes sense. Using recursive retriever.
-- Check figure extraction

Knowledge corpus - 1st Stage Embedding based retrival - docs - 2nd stage LLM based reranking - Docs.....
Load vectors into Database Qrand
1. Getting llmam parse to work. 
2. retrieval - query engines.


llamaparse - nice tabular extraction - which I need for meta. Speed is lsow and figures are not retirieved correctlt. 

In [1]:
!pip install -qU llama-index llama-parse

In [2]:
import os
import getpass

os.environ["LLAMA_CLOUD_API_KEY"] = getpass.getpass("LLamaParse API Key:")

In [3]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

In [4]:
# Uncomment if you are in a Jupyter Notebook - I did.
import nest_asyncio

nest_asyncio.apply()

In [5]:
from llama_parse import LlamaParse

parser = LlamaParse(
    result_type="markdown", # set it to markdown so preservess sturcture
    verbose=True,  # output detail logs
    language="en",
    num_workers=1,  #I will parse one worker since I only parse one. 
)

# Load the document

In [ ]:
#create a folder and then download and update its name:
!mkdir -p 'data/'
!wget 'https://d18rn0p25nwr6d.cloudfront.net/CIK-0001326801/c7318154-f6ae-4866-89fa-f0c589f2ee3d.pdf' -O 'data/meta_10k_filings.pdf'

In [6]:
#Initially I used SimpleDirectoryReader from llama_index.core but parser has a method to load data from a file.
#sync
documents = parser.load_data(["./data/meta_10k_filings.pdf"]) # keeping documents to mention corpus & remember there cna be multiple docs. 
print(f"Loaded {len(documents)} document(s)")

Parsing files: 100%|██████████| 1/1 [00:08<00:00,  8.12s/it]

Loaded 1 pages


In [8]:
print(type(documents), "\n") #list just like movies!!!
print(type(documents[0])) # type document
print(documents[0].text[:500]) # shows the markdown text of the document. Good.

<class 'list'> 

<class 'llama_index.core.schema.Document'>
## SECURITIES AND EXCHANGE COMMISSION UNITED STATES Washington, D.C. 20549

## FORM 10-K

(Mark One)

☒ ANNUAL REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934

For the fiscal year ended December or 31, 2023

☐ TRANSITION REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934

For the transition period from to

Commission File Number: 001-35551

Meta Platforms, Inc. Meta

(Exact name of registrant as specified in its charter)

Delaware 20-16650


## LlamaIndex Recursive Query Engine

In [9]:
from llama_index.core import Settings
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding

Settings.llm = OpenAI(temperature=0, model="gpt-3.5-turbo") # Will change to 4 in prod. 
Settings.embed_model = OpenAIEmbedding(model="text-embedding-3-small")

# Vanilla Parsing aka without parsing instructions

In [10]:
from llama_index.core.node_parser import MarkdownElementNodeParser

node_parser = MarkdownElementNodeParser(llm=OpenAI(model="gpt-3.5-turbo"), num_workers=8)

In [11]:
nodes = node_parser.get_nodes_from_documents(documents=[documents[0]])

129it [00:00, 37176.39it/s]
100%|██████████| 129/129 [00:34<00:00,  3.77it/s]


In [12]:
base_nodes, objects = node_parser.get_nodes_and_objects(nodes) 

# Parsing with Instructions

parsingInstruction10K = """
The provided document is a 10K filing that contains various sections of text and tables. Parse the document as follows:

Signature Table:

Locate the table with columns for Signature, Title, and Date.
Extract the data from each row of the table and create a structured Markdown table with the following columns: Name, Title, and Date.
For the Name column, remove any signature prefixes (e.g., '/s/' or '/s') and only include the actual name.
Preserve the original titles and dates as they appear in the table.
The resulting Markdown table should be formatted properly with pipes (|) separating the columns and dashes (-) separating the header row from the data rows.


Other Sections:

Identify and extract the main sections of the document, such as the introduction, business description, risk factors, financial statements, etc.
For each section, create a Markdown heading with the section title.
Extract the text content of each section and include it under the corresponding heading.
If a section contains subsections, create subheadings for each subsection and include the relevant text content.
Preserve the original formatting of the text, such as paragraphs, lists, and tables, as much as possible.


Output:

Combine the parsed signature table and the other sections into a single Markdown document.
Ensure that the signature table appears at the appropriate location within the document, typically towards the end.
Use proper Markdown syntax for headings, subheadings, paragraphs, lists, tables, and any other formatting elements.
The final output should be a well-structured Markdown representation of the entire 10K document, including the signature table and all other relevant sections.
"""

In [ ]:
"""The provided document contains a table listing signatures, titles, and dates. Extract the data from this table and create a Markdown table with the following columns: Name, Title, and Date. For the Name column, remove any signature prefixes (e.g., '/s/' or '/s') and only include the actual name. Preserve the original titles and dates as they appear in the image. The resulting Markdown table should be formatted properly with pipes (|) separating the columns and dashes (-) separating the header row from the data rows."""

In [51]:
parsingInstructionMeta = """The provided document contains a table listing signatures, titles, and dates. Extract the data from this table and create a Markdown table with the following columns: Name, Title, and Date. For the Name column, remove any signature prefixes (e.g., '/s/' or '/s') and only include the actual name. Preserve the original titles and dates as they appear in the image. The resulting Markdown table should be formatted properly with pipes (|) separating the columns and dashes (-) separating the header row from the data rows."""
withInstructionParsing = LlamaParse(
    result_type="markdown", parsing_instruction=parsingInstructionMeta
).load_data("/Users/acrobat/Documents/GitHub/AI-Engineering-Cohort-2/midterm/data/meta_10k_filings.pdf")

Started parsing the file under job_id dc418bf5-b0c0-4f66-a670-d82694b0e588
..................

In [65]:
print(type(withInstructionParsing), "\n") #list just like movies!!!
print(type(withInstructionParsing[0])) # type document
print(withInstructionParsing[0].text[:500]) # shows the markdown text of the document. Good.

<class 'list'> 

<class 'llama_index.core.schema.Document'>
| Name | Title | Date |
|------|-------|------|
| SECURITIES AND EXCHANGE COMMISSION UNITED STATES | Washington, D.C. 20549 | - |
| FORM 10-K | - | - |
| Meta Platforms, Inc. Meta | Delaware 20-1665019 | - |
| Class A Common Stock, $0.000006 par value | META | The Nasdaq Stock Market LLC |
---
| Name | Title | Date |
|------|-------|------|
|       |       |      |
|       |       |      |
|       |       |      |
|       |       |      |
|       |       |      |
---
| Name | Title | Date |
|---


In [52]:

target_page = 133

print(withInstructionParsing[0].text.split("\n---\n")[target_page])

| Name              | Title                                     | Date            |
|-------------------|-------------------------------------------|-----------------|
| Mark Zuckerberg   | Board Chair and Chief Executive Officer  | February 1, 2024 |
| Susan Li          | Chief Financial Officer                   | February 1, 2024 |
| Aaron Anderson    | Chief Accounting Officer                  | February 1, 2024 |
| Peggy Alford      | Director                                  | February 1, 2024 |
| Marc L. Andreessen| Director                                  | February 1, 2024 |
| Andrew W. Houston | Director                                  | February 1, 2024 |
| Nancy Killefer    | Director                                  | February 1, 2024 |
| Robert M. Kimmitt | Director                                  | February 1, 2024 |
| Sheryl K. Sandberg | Director                                 | February 1, 2024 |
| Tracey T. Travis  | Director                                  | Fe

In [66]:

target_page = 128

print(withInstructionParsing[0].text.split("\n---\n")[target_page])

| Name | Title | Date |
|------|-------|------|
| Directors, Executive Officers and Corporate Governance | The information required by this item is incorporated by reference to our Proxy Statement for the 2024 Annual Meeting of Stockholders to be filed with the SEC within 120 days of the fiscal year ended December 31, 2023. | - |
| Executive Compensation | The information required by this item is incorporated by reference to our Proxy Statement for the 2024 Annual Meeting of Stockholders to be filed with the SEC within 120 days of the fiscal year ended December 31, 2023. | - |
| Security Ownership of Certain Beneficial Owners and Management and Related Stockholder Matters | The information required by this item is incorporated by reference to our Proxy Statement for the 2024 Annual Meeting of Stockholders to be filed with the SEC within 120 days of the fiscal year ended December 31, 2023. | - |
| Certain Relationships and Related Transactions, and Director Independence | The informatio

In [62]:

target_page = 94

print(withInstructionParsing[0].text.split("\n---\n")[target_page])

| Name | Title | Date |
|------|-------|------|
| Net income | $39,098 | $23,200 | $39,370 |
| Depreciation and amortization | $11,178 | $8,686 | $7,967 |
| Share-based compensation | $14,027 | $11,992 | $9,164 |
| Deferred income taxes | $131 | ($3,286) | $609 |
| Impairment charges for facilities consolidation, net | $2,432 | $2,218 | — |
| Data center assets abandonment | ($224) | $1,341 | — |
| Other | $635 | $641 | ($127) |
| Accounts receivable | ($2,399) | $231 | ($3,110) |
| Prepaid expenses and other current assets | $559 | $162 | ($1,750) |
| Other assets | ($80) | ($106) | ($349) |
| Accounts payable | $51 | $210 | $1,436 |
| Partners payable | ($271) | $90 | ($12) |
| Accrued expenses and other current liabilities | $5,352 | $4,210 | $3,544 |
| Other liabilities | $624 | $886 | $941 |
| Purchases of property and equipment | ($27,266) | ($31,431) | ($18,690) |
| Proceeds relating to property and equipment | $221 | $245 | $123 |
| Purchases of marketable debt securities | ($2

In [67]:
nodes = node_parser.get_nodes_from_documents(withInstructionParsing)

143it [00:00, 51907.01it/s]
100%|██████████| 143/143 [00:28<00:00,  5.10it/s]


In [77]:
print(len(nodes))

428


In [68]:
base_nodes, objects = node_parser.get_nodes_and_objects(nodes) 

# Build the  Index (But where? I need this to be in Qdrand)

In [69]:
from llama_index.core import VectorStoreIndex

recursive_index = VectorStoreIndex(nodes=base_nodes+objects)

# Recursive Query Engine

In [14]:
!pip install -qU llama-index-postprocessor-flag-embedding-reranker git+https://github.com/FlagOpen/FlagEmbedding.git

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
fastembed 0.2.6 requires huggingface-hub<0.21,>=0.20, but you have huggingface-hub 0.22.2 which is incompatible.
fastembed 0.2.6 requires tokenizers<0.16.0,>=0.15.1, but you have tokenizers 0.19.1 which is incompatible.


In [ ]:

! pip install huggingface-hub==0.20.0
! pip install tokenizers==0.15.1
! pip install -qU llama-index-postprocessor-flag-embedding-reranker git+https://github.com/FlagOpen/FlagEmbedding.git

# Initialize the reranker - we'll be leveraging this repo to leverage our BAAI/bge-reranker-large.

In [70]:
from llama_index.postprocessor.flag_embedding_reranker import FlagEmbeddingReranker

reranker = FlagEmbeddingReranker(
    top_n=5, # retun 5 from 15
    model="BAAI/bge-reranker-large",
)

recursive_query_engine = recursive_index.as_query_engine(
    similarity_top_k=15,  #grab 15 top results
    node_postprocessors=[reranker],
    verbose=True
)

In [71]:
query = "What was the total value of 'Cash and cash equivalents' as of December 31, 2023?"
response = recursive_query_engine.query(query)

Retrieval entering d11e66f0-2353-49b9-b04a-919eddaea8be: TextNode
Retrieving from object TextNode with query What was the total value of 'Cash and cash equivalents' as of December 31, 2023?
Retrieval entering 4c132dfe-5f9a-4a90-b861-dc431eb0201a: TextNode
Retrieving from object TextNode with query What was the total value of 'Cash and cash equivalents' as of December 31, 2023?
Retrieval entering 1cd2e147-dca5-428a-a619-717dea634e79: TextNode
Retrieving from object TextNode with query What was the total value of 'Cash and cash equivalents' as of December 31, 2023?
Retrieval entering 5a3aca44-baf6-4000-bdbc-45e5cd0bee25: TextNode
Retrieving from object TextNode with query What was the total value of 'Cash and cash equivalents' as of December 31, 2023?
Retrieval entering 019a36f8-cce1-495f-8fd8-3b51866d0c82: TextNode
Retrieving from object TextNode with query What was the total value of 'Cash and cash equivalents' as of December 31, 2023?
Retrieval entering a6b5acd4-6e91-40fe-90d0-a058ce4

In [72]:
print(response) #yippee.

The total value of 'Cash and cash equivalents' as of December 31, 2023, was $41,862 million.


In [73]:
query = "What are the names of people with the director title at Meta?"
#query = "What are the names of people with the title = Director at Meta?"
response = recursive_query_engine.query(query)

Retrieval entering a40539d4-071f-4d27-a46f-e423d519121a: TextNode
Retrieving from object TextNode with query What are the names of people with the director title at Meta?
Retrieval entering 58615add-efe0-4cec-9d91-c35b39f1a9ca: TextNode
Retrieving from object TextNode with query What are the names of people with the director title at Meta?
Retrieval entering 8c1fa50e-8334-4e87-9288-3bd9b0a0401e: TextNode
Retrieving from object TextNode with query What are the names of people with the director title at Meta?
Retrieval entering 286c96d2-4d42-491a-a50e-c4befbd3d482: TextNode
Retrieving from object TextNode with query What are the names of people with the director title at Meta?
Retrieval entering bbe854f9-3032-4cda-a758-8193c2b1cca4: TextNode
Retrieving from object TextNode with query What are the names of people with the director title at Meta?
Retrieval entering 36e772df-fa7b-4b29-8736-2691fa4eaf42: TextNode
Retrieving from object TextNode with query What are the names of people with th

In [74]:
print(response)

Peggy Alford, Marc L. Andreessen, Andrew W. Houston, Nancy Killefer, Robert M. Kimmitt, Sheryl K. Sandberg, Tracey T. Travis, Tony Xu


In [75]:

query = "What is the value of total defferred tax liabilities in year 2023?"
response = recursive_query_engine.query(query)

Retrieval entering c008b47d-7f6d-4564-bc4b-5bd875d7b22e: TextNode
Retrieving from object TextNode with query What is the value of total defferred tax liabilities in year 2023?
Retrieval entering a4d826f2-dbe8-432d-9afb-21968026d1b3: TextNode
Retrieving from object TextNode with query What is the value of total defferred tax liabilities in year 2023?
Retrieval entering df68930f-ce97-4f09-b325-4d212959f700: TextNode
Retrieving from object TextNode with query What is the value of total defferred tax liabilities in year 2023?
Retrieval entering 3e7be0e5-6897-4c26-a892-90e4943f7747: TextNode
Retrieving from object TextNode with query What is the value of total defferred tax liabilities in year 2023?
Retrieval entering 5a3aca44-baf6-4000-bdbc-45e5cd0bee25: TextNode
Retrieving from object TextNode with query What is the value of total defferred tax liabilities in year 2023?
Retrieval entering 4c132dfe-5f9a-4a90-b861-dc431eb0201a: TextNode
Retrieving from object TextNode with query What is the

In [76]:
print(response)

The value of total deferred tax liabilities in year 2023 is ($11,028) million.


#######################

In [ ]:
meta_index = VectorStoreIndex.from_documents(documents)

In [2]:
import nest_asyncio

nest_asyncio.apply()

In [3]:
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex
from llama_index.llms.openai import OpenAI

from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.core.query_engine import SubQuestionQueryEngine

In [4]:
from llama_index.core import Settings

Settings.llm = OpenAI(temperature=0.2, model="gpt-3.5-turbo")

In [ ]:
#create a folder and then download and update its name:
!mkdir -p 'data/'
!wget 'https://d18rn0p25nwr6d.cloudfront.net/CIK-0001326801/c7318154-f6ae-4866-89fa-f0c589f2ee3d.pdf' -O 'data/meta_10k_filings.pdf'

In [ ]:
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex

reader = SimpleDirectoryReader(
    input_files=["./data/meta_10k_filings.pdf"]
)

documents = reader.load_data()
print(f"Loaded {len(documents)} pages")

In [5]:
meta_docs = SimpleDirectoryReader(
    input_files=["./data/meta_10k_filings.pdf"]
).load_data()
# uber_docs = SimpleDirectoryReader(
#     input_files=["./data/10k/uber_2021.pdf"]
# ).load_data()

In [6]:
meta_index = VectorStoreIndex.from_documents(meta_docs)

In [8]:
meta_engine = meta_index.as_query_engine(similarity_top_k=3)


In [12]:
query_engine_tools = [
    QueryEngineTool(
        query_engine=meta_engine,
        metadata=ToolMetadata(
            name="meta_10k",
            description=(
                "Provides information about Meta FORM 10-K"
            ),
        ),
    ),
    # QueryEngineTool(
    #     query_engine=uber_engine,
    #     metadata=ToolMetadata(
    #         name="uber_10k",
    #         description=(
    #             "Provides information about Uber financials for year 2021"
    #         ),
    #     ),
    # ),
]

s_engine = SubQuestionQueryEngine.from_defaults(
    query_engine_tools=query_engine_tools
)

In [15]:
response = s_engine.query(
"What was the total value of 'Cash and cash equivalents' as of December 31, 2023?"
    
)

Generated 1 sub questions.
[meta_10k] Q: What was the total value of 'Cash and cash equivalents' as of December 31, 2023?
[meta_10k] A: The total value of 'Cash and cash equivalents' as of December 31, 2023, was $65.40 billion.


In [20]:
response = s_engine.query(
"What are the names of people with the director title at Meta?"
    
)

Generated 1 sub questions.
[meta_10k] Q: What are the names of people with the director title at Meta?
[meta_10k] A: The names of people with the director title at Meta are not provided in the given context information.
